In [1]:
import pyodbc
from import_function import import_scoring_data, import_data
from features_by_customer_type import customer_type_features
conn = pyodbc.connect(dsn='VerticaProd')
x=120
ct='New'
response=['ABANDONED']

cont_features, bool_features, catag_features, cont_score_features, bool_score_features, catag_score_features=customer_type_features(ct)
scoring_data = "SELECT A.* FROM (SELECT * FROM  CTG_ANALYTICS_WS.SM_RETENTION_MODEL  WHERE   TAX_YEAR=2016 AND CUSTOMER_DEFINITION_ADJ='NEW TO TURBOTAX' )A inner join (SELECT DISTINCT CUSTOMER_KEY FROM  CTG_ANALYTICS_WS.SM_RETENTION_MODEL  WHERE   TAX_YEAR=2016 AND TAX_DAY=120) B ON A.CUSTOMER_KEY=B.CUSTOMER_KEY limit 100"
data = "SELECT * FROM (SELECT * FROM  CTG_ANALYTICS_WS.SM_RETENTION_MODEL  WHERE   TAX_YEAR=2015 AND  CUSTOMER_DEFINITION_ADJ IN ('NEW TO TURBOTAX')    )A   order by random() limit 100000"
df_no_pca = import_data(data, cont_features, bool_features,response, catag_features)
scoring_df=import_scoring_data(scoring_data, cont_score_features, bool_score_features,catag_score_features)





c:\python27\lib\site-packages\pandas\core\frame.py:2762: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


df_cont done
data_scaled done
just_dummies done
df_trans done
new feature list done


import_function.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df_bool.drop([f], axis=1, inplace=True)


In [2]:
from bin_cont_features import bin_pca
j='True'
response='ABANDONED'
df_pca=df_no_pca
dummy_pca, length_dict = bin_pca(df_pca,response, j)
print list(dummy_pca), dummy_pca.head()
print dict

['<=3615908.5', '<=120009.5', 'between 120009.5 and 3615908.5', '>3615908.5', 'between 3615908.5 and 4003514.0', '>=4003514.0']
['<=416258.0', '<=411002.0', 'between 411002.0 and 416258.0', '>416258.0', 'between 416258.0 and 3615908.5', '>=3615908.5']
['<=145911.0', '<=52889.5', 'between 52889.5 and 145911.0', '>145911.0', 'between 145911.0 and 707319.0', '>=707319.0']
['<=159625.5', '<=52889.5', 'between 52889.5 and 159625.5', '>159625.5', 'between 159625.5 and 3615908.5', '>=3615908.5']
['<=51663.5', '<=27104.0', 'between 27104.0 and 51663.5', '>51663.5', 'between 51663.5 and 1034523.5', '>=1034523.5']
['<=178714.0', '<=19390.0', 'between 19390.0 and 178714.0', '>178714.0', 'between 178714.0 and 1468240.0', '>=1468240.0']
['<=223649.5', '<=19237.0', 'between 19237.0 and 223649.5', '>223649.5', 'between 223649.5 and 1468240.0', '>=1468240.0']
['<=52889.5', '<=15513.5', 'between 15513.5 and 52889.5', '>52889.5', 'between 52889.5 and 968254.5', '>=968254.5']
['<=15021.0', '<=14331.0', '

In [3]:
response='ABANDONED'
ct='New'
from feature_clustering import feature_clustering
i='pca'
df_no_pca=dummy_pca
df_no_pca,y, plsca= feature_clustering(df_no_pca,response, i)




sample dataframe shape is:  (10000, 1303)
{130: 43.536962097363386, 70: 44.052367852524661, 170: 43.860634842550432, 110: 43.894662412630794, 50: 44.154334135309902, 150: 44.047370155991892, 90: 44.012823203625707, 190: 43.018003185356577}
the best # of components is: 50


In [4]:
from clustering import obs_clustering
clust='True'
df=df_no_pca
df, df_w_labels, df_w_clust=obs_clustering(df,response,clust)


(10000, 41)
42.031262812
42.2222341837
43.3298218151
15
14    9749
6     9601
0     9023
3     8362
7     8069
1     7180
4     6615
13    6480
12    6193
10    6170
9     6038
8     4572
5     4371
11    3825
2     3752
Name: LABELS, dtype: int64


In [5]:
from pca_dim_red import pca_code
i='False'
response='ABANDONED'
df_no_pca=df_w_labels
print list(df_no_pca)
df_pca, fitted_pca = pca_code(df_no_pca,response,i)



['LABELS', "COST_LAG5_pcl_'<=19390.0'", "COST_LAG6_pcl_'between 19237.0 and 223649.5'", "COST_LAG3_pcl_'>159625.5'", u'FED_FORM_TYPE_F1040A', 'PRS_SCORE_8.0', u'ENTRY_PAGE_GROUP_Microsite Pages', "AMOUNT_HOPE_CREDIT_pcl_'<=6.0'", u'CHANNEL_Paid Search - Brand', 'NUM_W2_-1', "AGE_TAXPAYER_pcl_'<=3.0'", "COST_LAG2_pcl_'>145911.0'", u'IMPORT_TYPE_Import', "COST_LAG7_pcl_'between 15513.5 and 52889.5'", 'NUM_W2_3.0', "COST_LAG14_pcl_'<=15021.0'", "COST_LAG2_pcl_'between 52889.5 and 145911.0'", "TOTAL_REVENUE_pcl_'between 69.230003356933594 and 71.730003356933594'", u'FED_FORM_TYPE_F1040', "COST_LAG7_pcl_'<=15513.5'", 'PRS_SCORE_10.0', "AGE_TAXPAYER_pcl_'between 3.0 and 27.5'", 'REJECT_COUNT_1.0', "COST_LAG14_pcl_'between 14331.0 and 15021.0'", 'PRS_SCORE_-1', "TOTAL_COST_pcl_'>=4003514.0'", 'IMPORT_TYPE_-1', "STATE_REVENUE_pcl_'<=30.994998931884766'", u'ENTRY_PAGE_GROUP_Homepage', u'START_SKU_200|Free TTO', "NON_CA_REFUND_TRANSFER_REVENUE_pcl_'>33.165000915527344'", "COST_LAG3_pcl_'between 

In [6]:
from get_arrays import get_arrays
i='False'
j='False'
dummy_pca=df_pca
response = 'ABANDONED'
x, y = get_arrays(dummy_pca, response, i, j)

In [7]:
from model_algorithm import algorithm
response = 'ABANDONED'
models, name = algorithm(x, y, response)

c:\python27\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Random_Forest 0.550185219389 0.346353715572 0.41004870546 0.565883046146 14494 3960 7547 3999
                                                         sig
CHANNEL_Paid Search - Brand                         0.055608
ENTRY_PAGE_GROUP_Homepage                           0.054180
IMPORT_TYPE_Import                                  0.048200
NUM_W2_2.0                                          0.045503
CHANNEL_Non-Campaign                                0.041432
FED_FORM_TYPE_F1040A                                0.033976
NON_CA_REFUND_TRANSFER_REVENUE_pcl_'>33.1650009...  0.033419
AGE_TAXPAYER_pcl_'<=3.0'                            0.033192
NUM_W2_3.0                                          0.031713
ENTRY_PAGE_GROUP_Other                              0.031374
PRS_SCORE_10.0                                      0.029032
COST_LAG3_pcl_'<=52889.5'                           0.028979
NUM_W2_-1                                           0.028928
COST_LAG7_pcl_'<=15513.5'                           

In [8]:
import pandas as pd
import numpy as np
from predicting_code import transform_to_pca, bin_pca_score_set, get_scoring_arrays, score_set_feature_selection, predict,get_scoring_arrays
defection=pd.DataFrame()
i, j, k=['False','True','False' ]
df_trans_pca = transform_to_pca(scoring_df, fitted_pca, i)
# print df_trans_pca
scoring_df_trans = bin_pca_score_set(df_trans_pca, length_dict, j)

print scoring_df_trans.shape, x.shape
a=list(x)
b=list(scoring_df_trans)
final=list(set(b).difference(a))
scoring_df_trans.drop(final, axis=1, inplace=True)
print scoring_df_trans.shape
x.drop(list(scoring_df_trans), axis=1, inplace=True)
print x.shape
c=list(x)

m=scoring_df_trans.shape[0]     
n= x.shape[1]
print  scoring_df_trans.shape, x.shape

zeros_df=pd.DataFrame(np.zeros((m, n)), columns=c)



scoring_df_trans.reset_index(['CUSTOMER_KEY'], inplace=True)

final_scoring_df=pd.concat([zeros_df,scoring_df_trans ], axis=1)
final_scoring_df.set_index('CUSTOMER_KEY', inplace=True)
print final_scoring_df.shape

scoring_df_trans=final_scoring_df
x_score, index=score_set_feature_selection(scoring_df_trans,plsca, k)
x_score.set_index('CUSTOMER_KEY', inplace=True)
df_p=predict(models,name,x_score, index)
defection=pd.concat([df_p, defection], axis=1)
print defection.head()


TOTAL_COST
dict list [4, 6, 6, 6, 6, 4, 4, 4, 6, 6, 4, 4, 4, 4, 4, 4, 6, 6, 6, 4, 6, 6, 4, 6, 6, 4, 4, 6, 4, 4, 4, 6, 6, 6, 4, 4, 4, 4, 4, 4, 6, 6, 4, 6, 4, 6, 6, 4, 6, 4, 6, 6, 6, 6, 6, 4, 6, 6, 6, 6, 4, 4, 6, 6, 4, 6, 6, 4, 6, 6, 4, 6, 6, 4, 6, 6, 4, 6, 4, 4, 6, 4, 6, 6, 6, 6, 6, 6, 4, 4, 4, 4, 4, 6, 4, 4, 4, 4, 4, 6, 6, 4, 4, 4, 4, 4, 4, 6, 6, 4, 6, 6, 4, 6, 6, 6, 6, 6, 4, 4, 4, 6, 6, 6, 6, 6, 4, 4, 4, 6, 6, 4, 4, 4, 6, 6, 4, 6, 4, 6, 6, 6, 6, 6, 6, 6, 6, 6, 4, 4, 6, 4, 4, 4, 6, 6, 4, 4, 6, 6, 6, 6, 6, 6, 4, 6, 4, 4, 6, 6, 6, 6, 4, 4, 4, 4, 4, 6, 6, 4, 6, 6, 4, 4, 4, 6, 6, 4, 4]
df_trans_pca[i] CUSTOMER_KEY
5811894             0.0
6224282             0.0
7427843             0.0
9588785             0.0
9824093             0.0
10532895            0.0
10637179            0.0
12061284            0.0
12066321            0.0
17543231            0.0
20736637            0.0
1010000273010014    0.0
1010000273100047    0.0
1010000273265356    0.0
1010000273270129    0.0
1010000273270251    0.

ValueError: Bin edges must be unique: array([ 0.,  0.,  0.,  0.,  0.])

In [ ]:
from data_intersection import scoring_data_intersection
data = scoring_data_intersection(df_no_pca, scoring_df)
# print data.head()
print df_cont